In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from glob import glob
sys.path.append('../../src')
sys.path.append('../../')
import classifier.Multiband as multiband


the filtering of the times must be done in the spine, not in each band. Spit all the times then slect the best one. 

In [2]:
# To see if the system regognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)


2023-07-07 00:56:50.977866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:50.978100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:50.983007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:50.983205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:50.983366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Find the different folds and train a model using the stored data.

In [3]:
survey = 'Gaia'
path = os.path.join('../02_CreateRecords/Folds/Fold_*/',)
folds = glob(path)
folds.sort()
folds

['../02_CreateRecords/Folds/Fold_1/',
 '../02_CreateRecords/Folds/Fold_2/',
 '../02_CreateRecords/Folds/Fold_3/']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [5]:
train_args = {
            'hidden_size_bands':[128, 128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':5,
            'num_threads':7,
            'batch_size':1024,
            'dropout':0.40,
            'lr':[[1e-3]*2, 1e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':0, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression'
            }
loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,   
                    'N_skip' : 8, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': True,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [ ]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../02_CreateRecords', '.').replace('/Folds/', '/Results/')

    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    

    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'  
    train_args_specific['path_scalers'] =  base_dir+'scalers.pkl'
    # Define the train args
    train_args = {**train_args, **train_args_specific}
    
    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    new.train_loop()


2023-07-07 00:56:51.009895: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 00:56:51.010327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:51.010524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 00:56:51.010664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Start training


2023-07-07 00:57:07.880832: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101


13/13 [==============================] - 2s 121ms/step - loss: 6186.6870 - Class_loss: 20.6152 - Pred_Radius_loss: 1.1346 - Pred_T_eff_loss: 0.0499 - Class_CentralAcc: 0.6429 - Class_CentralTop2: 0.8330 - Class_FinalAcc: 0.6615 - Class_FinalTop2: 0.8410 - Class_Final_FScore: 0.5151 - Pred_Radius_Masked_RMSE: 0.8030 - Pred_Radius_Masked_R2: 0.5984 - Pred_T_eff_Masked_RMSE: 0.1813 - Pred_T_eff_Masked_R2: 0.7913
              precision    recall  f1-score   support

         CEP       0.52      0.72      0.61      1367
  DSCT_SXPHE       0.60      0.21      0.31       880
     MIRA_SR       0.85      0.91      0.88      2200
        RRAB       0.68      0.62      0.65      2200
         RRC       0.58      0.73      0.65      2200
       T2CEP       0.00      0.00      0.00       436

    accuracy                           0.66      9283
   macro avg       0.54      0.53      0.51      9283
weighted avg       0.64      0.66      0.63      9283

{'R2': {'T_eff': 0.7663711238081204, 'Radius